<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install -q langchain-ollama
!pip install -q langchain
!pip install -q langchain_community
!pip install -q sentence-transformers
!pip install -qU langchain-google-genai

In [55]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_google_genai import ChatGoogleGenerativeAI
import re
import json
import subprocess
import time
import threading
import psycopg2
from psycopg2 import sql

In [56]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

Enter your postgresql url: ··········


In [57]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [58]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

In [48]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13344    0 13344    0     0  62077      0 --:--:-- --:--:-- --:--:-- 62355
>>> Downloading Linux amd64 CLI
############################################################################################# 100.0%
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [49]:
# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")

Ollama server is running in the background


In [50]:
!ollama pull llama3:latest

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIJFZhADPB4fUDfhLGKqOCrM2vPlYMgu8lGG3uJho+tHE

[GIN] 2024/08/20 - 14:50:25 | 200 |     123.403µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 6.7 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  30 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  50 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  58 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏  72 MB/4.7 GB                  pulling manifest 
pulling 

In [51]:
llm = OllamaLLM(model="llama3:latest")

In [59]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

Your task is to extract key information from a travel request.

    Request: "{travel_request}"

    Extract and return the result by vietnamese in the following format:
    {{
        "Type": "...",
        "District": "...",
        "City": "...",
        "Number_of_people": "...",
        "Price": "...",
        "Rating": "...",
        "Description": "..."
    }}

    Ensure that the "Type" is one of the following: "Hotel," "Restaurant," or "TouristAttraction." If one or two types are mentioned, return only those. If none are mentioned, include all three types.

    For any information not specified in the travel request, return `null`. Ensure that the JSON result is strictly valid JSON, with no extra text, comments, or parentheses.

    The "District" not include "quận" or "huyện". The "City" not include "thành phố" or "tỉnh".

In [60]:
template = """
    You are an AI travel suggestion chatbot. Your task is to analyze the Request:

    Request: "{travel_request}"

    Extract the following 7 pieces of information not in any specific order: Type, District, City, Number_of_people, Price, Rating, and Description.
    Return the result by vietnamese in the following format:
    {{
        "Type": "...",
        "District": "...",
        "City": "...",
        "Number_of_people": "...",
        "Price": "...",
        "Rating": "...",
        "Description": "..."
    }}

    Ensure that the "Type" is one of the following: "Hotel," "Restaurant," or "TouristAttraction." If one or two types are mentioned, return only those. If none are mentioned, include all three types.

    For any information not specified in the travel request, return `null`. Ensure that the JSON result is strictly valid JSON, with no extra text, comments, or parentheses.

    The "District" not include "quận" or "huyện". The "City" not include "thành phố" or "tỉnh".

    The "Price" should be an integer. The "Rating" should be a float.
    """
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

In [ ]:
query = """
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!
"""
response = chain.invoke(query)
cleaned_json_str = re.search(r'\{.*?\}', response, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)
result_dict

In [62]:
query = """
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!
"""
response = chain.invoke(query)
response_gemini = str(response.content)
cleaned_json_str = re.search(r'\{.*?\}', response_gemini, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)
result_dict

{'Type': 'Hotel',
 'District': 'Cầu Giấy',
 'City': 'Hà Nội',
 'Number_of_people': None,
 'Price': None,
 'Rating': None,
 'Description': 'chất lượng dịch vụ cao cấp'}

In [64]:
import psycopg2
from psycopg2 import sql

try:
    # Connect to PostgreSQL database
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    conn.commit()

    cur.execute("""
    DROP SCHEMA IF EXISTS travel_database CASCADE;
    CREATE SCHEMA IF NOT EXISTS travel_database;
    SET search_path TO travel_database, public;  -- Use public where PostGIS is installed

    -- Create a custom type for the Address structure
    CREATE TYPE Address AS (
        street TEXT,
        district TEXT,
        city TEXT
    );

    -- Create the Hotel table with an embedding vector column
    CREATE TABLE Hotel (
        hotel_id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        address Address,
        location GEOMETRY(POINT),  -- PostGIS geography type for location data
        rating DECIMAL(2, 1),
        description TEXT,
        embedding_description vector(768),  -- Using the vector extension for 768-dimensional embeddings
        img_url JSON,
        comments JSON
    );

    -- Create an index on the district of the Address and Rating columns for the Hotel table
    CREATE INDEX idx_hotel_address_rating ON Hotel(((address).district), rating);

    -- Create the Price table for the Hotel table
    CREATE TABLE HotelPrice (
        hprice_id SERIAL PRIMARY KEY,
        hotel_id INT REFERENCES Hotel(hotel_id) ON DELETE CASCADE,
        room_type VARCHAR(255),
        capacity INT,
        price INT NOT NULL
    );

    -- Create an index on the hotel_id for the HotelPrice table
    -- CREATE INDEX hotel_idx_address_district ON Hotel(((address).district));
    -- CREATE INDEX hotel_idx_rating ON Hotel(rating);
    -- CREATE INDEX hotel_idx_location ON Hotel USING GIST(location);


    -- Create the TouristAttraction table with an embedding vector column
    CREATE TABLE TouristAttraction (
        attraction_id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        address Address,
        location GEOMETRY(POINT),  -- PostGIS geography type for location data
        attraction_type VARCHAR(255),
        rating DECIMAL(2, 1),
        tour_duration INTERVAL,
        description TEXT,
        embedding_description vector(768),  -- Using the vector extension for 768-dimensional embeddings
        img_url JSON,
        comments JSON
    );

    -- Create an index on the district of the Address and Rating columns for the TouristAttraction table
    -- CREATE INDEX attraction_idx_address_district ON TouristAttraction(((address).district));
    -- CREATE INDEX attraction_idx_rating ON TouristAttraction(rating);
    -- CREATE INDEX attraction_idx_location ON TouristAttraction USING GIST(location);


    -- Create the Price table for the TouristAttraction table
    CREATE TABLE AttractionPrice (
        aprice_id SERIAL PRIMARY KEY,
        attraction_id INT REFERENCES TouristAttraction(attraction_id) ON DELETE CASCADE,
        ticket_type VARCHAR(255),
        number_people INT,
        price INT NOT NULL
    );

    -- Create an index on the attraction_id for the AttractionPrice table
    -- CREATE INDEX idx_attractionprice_attractionid ON AttractionPrice(attraction_id);

    -- Create the Restaurant table with an embedding vector column
    CREATE TABLE Restaurant (
        res_id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        address Address,
        location GEOMETRY(POINT),  -- PostGIS geography type for location data
        rating DECIMAL(2, 1),
        description TEXT,
        embedding_description vector(768),  -- Using the vector extension for 768-dimensional embeddings
        img_url JSON,
        comments JSON
    );

    -- Create an index on the district of the Address and Rating columns for the Restaurant table
    -- CREATE INDEX res_idx_address_district ON Restaurant(((address).district));
    -- CREATE INDEX res_idx_rating ON Restaurant(rating);
    -- CREATE INDEX res_idx_location ON Restaurant USING GIST(location);
    """)
    conn.commit()

except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()

finally:
    # Close cursor and connection
    cur.close()
    conn.close()


In [65]:
def get_embedding(text):
    """Trả về embedding của văn bản sử dụng mô hình SentenceTransformer."""
    embedding = model.encode(text)
    return embedding.tolist()

# Dữ liệu mẫu
hotels = [
    ("Khách sạn A", "Khách sạn cao cấp gần trung tâm Hà Nội, với các tiện nghi hiện đại và dịch vụ khách hàng xuất sắc.", 4.5),
    ("Khách sạn B", "Khách sạn tiện nghi và thoải mái, lý tưởng cho những ai tìm kiếm sự yên tĩnh và thư giãn sau một ngày dài.", 3.8),
    ("Khách sạn C", "Khách sạn giá rẻ nhưng chất lượng tốt, phù hợp cho các du khách với ngân sách hạn chế.", 4.2),
    ("Khách sạn D", "Khách sạn sang trọng với dịch vụ 5 sao, bao gồm spa, bể bơi và nhà hàng cao cấp.", 4.9),
    ("Khách sạn E", "Khách sạn gần sân bay và thuận tiện di chuyển, cung cấp dịch vụ đưa đón và hỗ trợ 24/7.", 4.1)
]

attractions = [
    ("Tháp Rùa", "Một biểu tượng nổi tiếng của Hà Nội, nằm trên hồ Hoàn Kiếm, rất phổ biến với du khách và người dân địa phương.", "Di tích lịch sử và điểm tham quan nổi bật", 4.7),
    ("Chùa Một Cột", "Chùa cổ kính với kiến trúc độc đáo, nổi tiếng với hình dáng như một bông sen và nằm trên một cột duy nhất.", "Di tích văn hóa và tâm linh", 4.6),
    ("Lăng Bác", "Lăng chủ tịch Hồ Chí Minh, nơi lưu giữ di hài của Hồ Chí Minh, một điểm đến quan trọng và thiêng liêng ở Hà Nội.", "Nơi thờ cúng và lưu giữ di sản", 4.8),
    ("Văn Miếu Quốc Tử Giám", "Trường đại học cổ xưa nhất của Việt Nam, nơi thờ các vị hiền triết và là một biểu tượng của giáo dục truyền thống.", "Di tích giáo dục và văn hóa", 4.9),
    ("Hồ Tây", "Hồ lớn nhất tại Hà Nội, là nơi lý tưởng cho các hoạt động giải trí, đi dạo và thư giãn bên bờ hồ xanh mát.", "Khu vực giải trí và thư giãn", 4.4)
]

restaurants = [
    ("Nhà hàng A", "Nhà hàng phục vụ các món ăn địa phương với hương vị đậm đà, nổi tiếng với món bún chả và phở.", 4.2),
    ("Nhà hàng B", "Nhà hàng phục vụ món Âu và Á, với thực đơn đa dạng và phong cách ẩm thực quốc tế.", 3.9),
    ("Nhà hàng C", "Nhà hàng hải sản tươi ngon, chuyên cung cấp các món hải sản tươi sống và món đặc sản vùng biển.", 4.3),
    ("Nhà hàng D", "Nhà hàng sang trọng với không gian ấm cúng, phù hợp cho các bữa tối lãng mạn và sự kiện đặc biệt.", 4.6),
    ("Nhà hàng E", "Nhà hàng nhanh và tiện lợi, phục vụ các món ăn nhanh và đồ uống, lý tưởng cho bữa trưa gọn gàng.", 4.0)
]


# Kết nối đến cơ sở dữ liệu PostgreSQL
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()
cur.execute("""SET search_path TO travel_database, public;""")

# Chèn dữ liệu vào bảng Hotel
for name, description, rating in hotels:
    embedding = get_embedding(description)
    cur.execute("""
        INSERT INTO Hotel (name, address, location, rating, description, embedding_description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, %s, %s, %s);
    """, (name, rating, description, embedding, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Chèn dữ liệu vào bảng TouristAttraction
for name, description, attraction_type, rating in attractions:
    embedding = get_embedding(description)
    cur.execute("""
        INSERT INTO TouristAttraction (name, address, location, attraction_type, rating, tour_duration, description, embedding_description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, NULL, %s, %s, %s, %s);
    """, (name, attraction_type, rating, description, embedding, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Chèn dữ liệu vào bảng Restaurant
for name, description, rating in restaurants:
    embedding = get_embedding(description)
    cur.execute("""
        INSERT INTO Restaurant (name, address, location, rating, description, embedding_description, img_url, comments)
        VALUES (%s, ROW('0', 'Cầu Giấy', 'Hà Nội'), ST_Point(105.8498, 21.0285), %s, %s, %s, %s, %s);
    """, (name, rating, description, embedding, json.dumps({"img": "url"}), json.dumps({"comments": "none"})))

# Commit các thay đổi
conn.commit()

# Đóng kết nối và con trỏ
cur.close()
conn.close()

In [47]:
# Gợi ý cho tôi khách sạn ở Cầu Giấy, Hà Nội chất lượng 5 sao giành cho 2 người giá khoảng 10 triệu đồng với chất lượng dịch vụ cao cấp!
query = input("Nhập câu truy vấn của bạn: ")

Nhập câu truy vấn của bạn: Gợi ý cho tôi khách sạn ở Cầu Giấy, Hà Nội chất lượng 5 sao giành cho 2 người giá khoảng 10 triệu đồng với chất lượng dịch vụ cao cấp!


In [46]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Đặt schema hiện tại
cur.execute("SET search_path TO travel_database, public;")

# Tạo vector cho mô tả truy vấn


# Thành phố cần tìm
response = chain.invoke(query)
cleaned_json_str = re.search(r'\{.*?\}', response, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)


embedding_query = model.encode(result_dict['Description'])
embedding_query_list = embedding_query.tolist()

# Kiểm tra xem bảng Hotel có tồn tại không
cur.execute("SELECT * FROM information_schema.tables WHERE table_name = 'hotel';")
if not cur.fetchone():
    print("Bảng 'Hotel' không tồn tại trong schema 'travel_database'.")
else:
    # Thực hiện truy vấn để tìm 2 khách sạn có độ tương đồng cao nhất
    cur.execute("""
        WITH query_embedding AS (
            SELECT %s::vector(768) AS query_vector
        ),
        similarity_scores AS (
            SELECT
                h.hotel_id,
                h.name,
                h.address,
                h.rating,
                h.description,
                (1 - (h.embedding_description <=> query_vector)) AS similarity
            FROM
                Hotel h
                CROSS JOIN query_embedding
            WHERE
                (h.address).city = %s
            ORDER BY
                similarity DESC
            LIMIT 2
        )
        -- Chọn 2 khách sạn có độ tương đồng cao nhất
        SELECT
            hotel_id,
            name,
            address,
            rating,
            description,
            similarity
        FROM
            similarity_scores;
    """, (embedding_query_list, result_dict['City']))

    # Lấy kết quả và in ra
    rows = cur.fetchall()

    print(f"Yêu cầu người dùng: {query}")
    for row in rows:
        print(f"HotelID: {row[0]}")
        print(f"Name: {row[1]}")
        print(f"Address: {row[2]}")
        print(f"Rating: {row[3]}")
        print(f"Description: {row[4]}")
        print(f"Similarity: {row[5]}")
        print("-" * 40)

    # print(result_dict)
# Đóng kết nối
cur.close()
conn.close()

Yêu cầu người dùng: Khách sạn giá rẻ nhưng chất lượng tốt, phù hợp cho các du khách với ngân sách hạn chế.
HotelID: 3
Name: Khách sạn C
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.2
Similarity: 1.0
----------------------------------------
HotelID: 8
Name: Khách sạn C
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.2
Similarity: 1.0
----------------------------------------


In [67]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Đặt schema hiện tại
cur.execute("SET search_path TO travel_database, public;")

# Tạo vector cho mô tả truy vấn


# Thành phố cần tìm
response = chain.invoke(query)
response_gemini = str(response.content)
cleaned_json_str = re.search(r'\{.*?\}', response_gemini, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)


embedding_query = model.encode(result_dict['Description'])
embedding_query_list = embedding_query.tolist()

# Kiểm tra xem bảng Hotel có tồn tại không
cur.execute("SELECT * FROM information_schema.tables WHERE table_name = 'hotel';")
if not cur.fetchone():
    print("Bảng 'Hotel' không tồn tại trong schema 'travel_database'.")
else:
    # Thực hiện truy vấn để tìm 2 khách sạn có độ tương đồng cao nhất
    cur.execute("""
        WITH query_embedding AS (
            SELECT %s::vector(768) AS query_vector
        ),
        similarity_scores AS (
            SELECT
                h.hotel_id,
                h.name,
                h.address,
                h.rating,
                h.description,
                (1 - (h.embedding_description <=> query_vector)) AS similarity
            FROM
                Hotel h
                CROSS JOIN query_embedding
            WHERE
                (h.address).city = %s
            ORDER BY
                similarity DESC
            LIMIT 2
        )
        -- Chọn 2 khách sạn có độ tương đồng cao nhất
        SELECT
            hotel_id,
            name,
            address,
            rating,
            description,
            similarity
        FROM
            similarity_scores;
    """, (embedding_query_list, result_dict['City']))

    # Lấy kết quả và in ra
    rows = cur.fetchall()

    print(f"Yêu cầu người dùng: {query}")
    for row in rows:
        print(f"HotelID: {row[0]}")
        print(f"Name: {row[1]}")
        print(f"Address: {row[2]}")
        print(f"Rating: {row[3]}")
        print(f"Description: {row[4]}")
        print(f"Similarity: {row[5]}")
        print("-" * 40)

    print(result_dict)
# Đóng kết nối
cur.close()
conn.close()

Yêu cầu người dùng: 
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!

HotelID: 1
Name: Khách sạn A
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.5
Description: Khách sạn cao cấp gần trung tâm Hà Nội, với các tiện nghi hiện đại và dịch vụ khách hàng xuất sắc.
Similarity: 0.23988989521233273
----------------------------------------
HotelID: 4
Name: Khách sạn D
Address: (0,"Cầu Giấy","Hà Nội")
Rating: 4.9
Description: Khách sạn sang trọng với dịch vụ 5 sao, bao gồm spa, bể bơi và nhà hàng cao cấp.
Similarity: 0.19820067049122325
----------------------------------------
{'Type': 'Hotel', 'District': 'Cầu Giấy', 'City': 'Hà Nội', 'Number_of_people': None, 'Price': None, 'Rating': None, 'Description': 'chất lượng dịch vụ cao cấp'}
